Test running sixdb in Python3 interactively

In [ ]:
# In case there is a bug with auto completion of paths, temp bugfix according to
# https://github.com/ipython/ipykernel/issues/299
%config Completer.use_jedi=False 

# add location of SixDeskDB for Python 3+ to module search path
import sys
# (example) set path to your sixdeskdb folder
sys.path.append("/afs/cern.ch/work/m/mtitze/hl-lhc/git/SixDeskDB/") 

import sixdeskdb as sdb

%config InlineBackend.figure_format = 'retina'  # sharp plots

In [ ]:
# to update functions if they are modified externally
%load_ext autoreload
%autoreload 2

In [ ]:
# set a name for the sixdesk SQL database to be analyzed.
study_name = '/afs/cern.ch/work/m/mtitze/hl-lhc/testbed/test_study/lhc_b1_inj_2016_qp_14_ioct_20_emit_3.57_2.87.db'
#study_name = '/afs/cern.ch/work/m/mtitze/hl-lhc/testbed/test_study/lhc_b1_inj_2016_qp_14_ioct_20_emit_3.57_2.87_original.db'

In [ ]:
study_database = sdb.SixDeskDB(dbname=study_name)

In [ ]:
study_database.inspect_results()

In [ ]:
study_database.plot_survival_2d(seed=3)

### new method

In [ ]:
# run sixdesk with a set of unequal emittances
emittances_x = [1.11, 1.53]
emittances_y = [3.48, 2.71, 2.42]

emittances_x = [3.57]
emittances_y = [2.87]

method = 1

sdb.RunDaVsTurns_ue(study_database, turnstep=100, emitx=emittances_x, emity=emittances_y, method=method)

### old method

In [ ]:
# check if da table exists in database. In case of problems, remove table da_vst
# in SQL database (e.g. with the script below)
study_database.check_table('da_vst')

In [ ]:
# run original sixdesk with a set of unequal emittances as countercheck. 
emittances_x = [3.57]
emittances_y = [2.87]

# define default values
force      = True # set to true to check calculation speed
turnstep   = 100
outfile    = False
outfileold = False
outfilefit = False
davstfit   = False
fitdat     = 'dawsimp'
fitdaterr  = 'dawsimperr'
fitndrop   = 25
fitskap    = -5.0
fitekap    = 5.0
fitdkap    = 0.01
     

turnstep= 100

outfile=True
outfileold=True 
outfilefit=True
   
davstfit=False # True may lead to a problem

sdb.RunDaVsTurns(study_database ,force, outfile, outfileold, turnstep, davstfit, fitdat, fitdaterr,
                    fitndrop, fitskap, fitekap, fitdkap, outfilefit, emitx=emittances_x[0], emity=emittances_y[0])

In [ ]:
# if necessary, drop tables
import itertools

import sqlite3

def drop_from_sql_database(filename, remove=['da_vst', 'fit_parameters'], verbose=True):
    '''
    This is a temporarily solution. We can remove tables from a sixdesk database
    in order to force/ensure that the values are re-computed if there is a change.
    '''
    if verbose:
        print ('Dropping {} from SQL database\n {}'.format(remove, filename))
    
    connection = sqlite3.connect(filename)
    cursor = connection.cursor()
    for table_name in remove:
        try:
            cursor.execute("DROP TABLE {}".format(table_name))
        except:
            print ('Problem with table {}'.format(table_name))
            continue
    connection.close()

In [ ]:
#to_remove = []
#for ex, ey in itertools.product(emittances_x, emittances_y):
#    to_remove.append('da_vst_{}_{}'.format(ex, ey).replace('.', ''))

#drop_from_sql_database(study_name, remove=to_remove)
drop_from_sql_database(study_name)